Streaming tables
 Materialized views
views 

In [0]:
%python

import dlt

In [0]:
%python
spark.sql("USE CATALOG shopping")

@dlt.table(
    comment="Order Bronze table",
    table_properties={"quality": "bronze"}
)
def order_bronze():
    df = spark.readStream.table("shopping.bronze.orders_raw")
    return df
     

In [0]:
%python
spark.sql("USE CATALOG shopping")
@dlt.table(
    comment="Customer Bronze table",
    table_properties={"quality": "bronze"},
     name="customer_bronze"
)

def customer_bronze():
         df = spark.readStream.table("shopping.bronze.customer_raw")
         return df
     

In [0]:
%python
@dlt.view(
    comment="Joined View"
)
def joined_vw():
    df_c = spark.read.table("Live.customer_bronze")
    df_o = spark.read.table("Live.order_bronze")
    df_join = df_c.join(df_o, how="left_outer", on=df_c.c_custkey == df_o.o_orderkey)
    return df_join

In [0]:
%python
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.functions import current_timestamp


In [0]:
%python
@dlt.table(
    comment="Joined table",
    table_properties={"quality": "silver"},
    name="joined_silver"
)
def joined_silver():
     df = spark.read.table("LIVE.joined_vw").withColumn("_InsertDate", current_timestamp())
     return df

In [0]:
%python
@dlt.table(
    comment="Orders Aggregated table",
    table_properties={"quality": "gold"}
)
def Orders_aggregated_gold():
    df_final = (spark.read.table("LIVE.joined_silver")
                .groupBy("c_mktsegment")
                .count()
                .withColumn("_InsertDate", current_timestamp()))
    return df_final

In [0]:
select * from  shopping.Orders_aggregated_gold